In [20]:
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
import tensorflow as tf

Now we will load the model that we saved earlier on the basis of best val_loss value

In [21]:
model = load_model('cnn.h5')

We use a haarcascade to detect faces. This particular haarcascade performs faster

In [22]:
face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [26]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cap=cv2.VideoCapture(0)
while True:
    ret,im=cap.read()
    if not ret:
        continue
    gray_img= cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(im)


    for (x,y,w,h) in faces_detected:
        face_img = im[y:y+h, x:x+h]
        face_img=cv2.resize(face_img,(100,100))
        normalized=face_img/255.0
        reshaped=np.reshape(normalized,(1,100,100,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        
        
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
    cv2.imshow('Mask Detector',im)
    key= cv2.waitKey(1) & 0xFF 
    if key==27:
        break
            
cap.release()
cv2.destroyAllWindows()

Since the model is trained only using white face masks , it is more sensitive towards white masks and not any darker colored masks.